In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install yahoo_fin --upgrade --no-cache-dir

In [ ]:
# DOLLAR COST AVERAGING
from datetime import date, timedelta, datetime

######## Select ticker, initial date, final date, initial amount, periodic amount to invest and periodicity in week days
ticker = "BND"
initial_date = date(2017,1,1)
final_date = date.today()
#final_date = date(1995,9,1)
initial_amount = 0
periodic_amount = 4000
week_days = 20
######## Select ticker, initial date, final date, initial amount, amount to purchases and periodicity
import numpy as np
from datetime import date, timedelta, datetime
import yfinance as yf  
import plotly.graph_objects as go
import pandas as pd

DF0 = yf.download(ticker,initial_date,final_date)
DF0['Date'] = DF0.index
Date_Purchases=[]
Values_Purchases=[]
Total_Shares = []
total_invested = 0
num_purchases = 0
j=-1
num_shares=0
def bbands(Precio,ventana,sigmas):
  """ returns average, upper band, and lower band"""
  Ave = Precio.rolling(window=ventana, center=False).mean()
  Sd = Precio.rolling(window=ventana,center=False).std() 
  Upband = Ave + Sd*sigmas
  Dnband = Ave - Sd*sigmas
  return np.round(Ave,3), np.round(Upband,3), np.round(Dnband,3)
for i in DF0.index:
  j+=1
  if (j%week_days == 0): 
    if (j==0): 
      num_shares += round((initial_amount + periodic_amount) /pd.to_numeric(DF0['Open'])[j],1)
      total_invested += round(initial_amount + periodic_amount,1)
    else: 
      num_shares += round( periodic_amount /pd.to_numeric(DF0['Open'])[j],1)
      total_invested += round(periodic_amount,1)
    Total_Shares.append(num_shares)
    num_purchases += 1
    Date_Purchases.append(DF0['Date'][j])
    Values_Purchases.append(pd.to_numeric(DF0['Open'])[j])
bb_avg, bb_upper, bb_lower = bbands(DF0.Open,20,2)
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = go.Figure(data=[go.Candlestick(x=DF0['Date'],
                open=DF0['Open'],
                high=DF0['High'],
                low=DF0['Low'],
                close=DF0['Close'])])
fig.update_layout(title_text="Dollar Cost Averaging for "+ticker,title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price per Share for "+ticker)
fig.add_trace(go.Scatter(x=DF0['Date'], y=bb_avg, name="Running Average"))
fig.add_trace(go.Scatter(x=DF0['Date'], y=bb_upper, name="2 Sigma Upper"))
fig.add_trace(go.Scatter(x=DF0['Date'], y=bb_lower, name="2 Sigma Lower"))
fig.add_trace(go.Scatter(x=Date_Purchases, y=Values_Purchases, name="Purchase",mode='markers',marker_symbol='circle',marker_color='black'))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()
final_value = round(DF0['Close'][-1]*Total_Shares[-1],1)
months =  (final_date.year - initial_date.year) * 12 + final_date.month - initial_date.month
print("Number of Months = ",months,", Total Number of Purchases:", num_purchases)
rend = round(100*(final_value/total_invested-1),1)
print("Final Price for "+ticker+" = ",round(DF0['Close'][-1],1),", Final Number of Shares: ", round(Total_Shares[-1],1))
print("Total Invested:",round(total_invested,1),", Final Value:",final_value)
print("Total Return:", rend,"%, Average of Periodic Purchases  = ", round(total_invested/months,1),"Dollars")